In [14]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [15]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from keras.utils import to_categorical

In [16]:
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense, LSTM, Embedding,Bidirectional
from keras.layers import Embedding, Conv1D, GRU, Dense
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Conv1D, MaxPooling1D

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!pip install text_hammer
import text_hammer as th

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.12.3
    Uninstalling beautifulsoup4-4.12.3:
      Successfully uninstalled beautifulsoup4-4.12.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.37 requires beautifulsoup4>=4.11.1, but you have beautifulsoup4 4.9.1 which is incompatible.


In [ ]:
# Load the dataset
#df = pd.read_csv('tweet_emotions.csv')
#df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [17]:
import pandas as pd# importing the dataset
df_train = pd.read_csv('train.txt', header =None, sep =';', names = ['Input','Sentiment'], encoding='utf-8')

df_test = pd.read_csv('test.txt', header = None, sep =';', names = ['Input','Sentiment'],encoding='utf-8')

df_val = pd.read_csv('val.txt',header=None,sep=';', names=['Input','Sentiment'], encoding='utf-8')

df = pd.concat([df_train, df_test, df_val], ignore_index=True)

In [18]:
# Apply label encoding to convert the generalized sentiment categories into numerical labels
label_encoder = LabelEncoder()
df['Sentiment'] = label_encoder.fit_transform(df['Sentiment'])

# Optionally, to see the mapping of sentiments to encoded labels:
sentiment_to_encoded = {label: encoding for encoding, label in enumerate(label_encoder.classes_)}
print(sentiment_to_encoded)

{'anger': 0, 'fear': 1, 'joy': 2, 'love': 3, 'sadness': 4, 'surprise': 5}


In [19]:
import re
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Expanding Contractions
def cont_exp(x):
    # A simple example of mapping contractions to their expansions
    contractions = {"you're": "you are", "i'm": "i am", "he's": "he is"}
    for key in contractions:
        x = x.replace(key, contractions[key])
    return x

def text_preprocessing(df, col_name):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    def clean_text(text):
        text = text.lower()  # Lowercase text
        text = cont_exp(text)  # Expand contractions

        # Remove emails
        text = re.sub(r'\S*@\S*\s?', '', text)
        # Remove HTML tags
        text = re.sub(r'<.*?>', '', text)
        # Remove special characters
        text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
        # Remove accented characters
        text = re.sub(r'[\xc3\xa1\xc3\xa9\xc3\xad\xc3\xb3\xc3\xba\xc3\xb1\xc3\x81\xc3\x89\xc3\x8d\xc3\x93\xc3\x9a\xc3\x91]','',text)

        # Tokenization and lemmatization
        tokens = word_tokenize(text)
        tokens = [lemmatizer.lemmatize(word) for word in tokens if not word in stop_words]  # Remove stopwords and lemmatize

        return ' '.join(tokens)  # Join back into a single string

    df[col_name] = df[col_name].apply(lambda x: clean_text(str(x)))
    return df

# Assuming df is your DataFrame and 'Input' is the column you want to preprocess
df_new = text_preprocessing(df, 'Input')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
df_new.head()

,Input,Sentiment
0,didnt feel humiliated,4
1,go feeling hopeless damned hopeful around some...,4
2,im grabbing minute post feel greedy wrong,0
3,ever feeling nostalgic fireplace know still pr...,3
4,feeling grouchy,0


In [21]:
df_new['Sentiment'].unique()

array([4, 0, 3, 5, 1, 2])

In [22]:
df_cat = to_categorical(df.Sentiment.values)
df_cat

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]], dtype=float32)

In [23]:
num_words = 10000 # this means 10000 unique words can be taken
tokenizer=Tokenizer(num_words,lower=True)
tokenizer.fit_on_texts(df_new['Input'])

In [24]:
max_seq_length = 100  # Set your desired sequence length
sequences = tokenizer.texts_to_sequences(df_new['Input'])
X = pad_sequences(sequences, maxlen = max_seq_length, padding='post', truncating='post')

In [25]:
corpus = [word_tokenize(text) for text in df_new['Input']]

In [ ]:
import gensim.downloader as api
glove_gensim  = api.load('glove-wiki-gigaword-100') #100 dimension

vector_size = 100
gensim_weight_matrix = np.zeros((num_words ,vector_size))

for word, index in tokenizer.word_index.items():
    if index < num_words:  # Consider only words within the desired vocabulary size
        try:
            # Attempt to retrieve the word vector from GloVe
            vector = glove_gensim[word]
            gensim_weight_matrix[index] = vector
        except KeyError:
            # If the word is not found, the vector remains all zeros
            pass  # This line is optional as the matrix is already initialized with zeros

print(gensim_weight_matrix.shape)

EMBEDDING_DIM = 100
class_num = 6

# Model Definition
model = Sequential()
model.add(Embedding(input_dim = num_words, output_dim = EMBEDDING_DIM, input_length= X.shape[1], weights = [gensim_weight_matrix],trainable = False))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(100,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(200,return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(100,return_sequences=False)))
model.add(Dense(6, activation = 'softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

(10000, 100)


In [28]:
import gensim.downloader as api
glove_gensim  = api.load('glove-wiki-gigaword-100') #100 dimension

vector_size = 100
gensim_weight_matrix = np.zeros((num_words ,vector_size))

for word, index in tokenizer.word_index.items():
    if index < num_words:  # Consider only words within the desired vocabulary size
        try:
            # Attempt to retrieve the word vector from GloVe
            vector = glove_gensim[word]
            gensim_weight_matrix[index] = vector
        except KeyError:
            # If the word is not found, the vector remains all zeros
            pass  # This line is optional as the matrix is already initialized with zeros

print(gensim_weight_matrix.shape)

EMBEDDING_DIM = 100
class_num = 6

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=EMBEDDING_DIM, input_length=X.shape[1], weights=[gensim_weight_matrix], trainable=False))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(200, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(100, return_sequences=False)))
model.add(Dense(6, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

(10000, 100)


In [30]:
#Split the data
X_train, X_test, y_train, y_test = train_test_split(X, df_cat, test_size=0.2, random_state=42)

# Train and Evaluate the Model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.2)
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

Epoch 1/10
400/400 [==============================] - 447s 1s/step - loss: 1.1262 - accuracy: 0.5852 - val_loss: 0.7703 - val_accuracy: 0.7316
Epoch 2/10
400/400 [==============================] - 452s 1s/step - loss: 0.7083 - accuracy: 0.7456 - val_loss: 0.5018 - val_accuracy: 0.8194
Epoch 3/10
400/400 [==============================] - 451s 1s/step - loss: 0.5198 - accuracy: 0.8126 - val_loss: 0.3640 - val_accuracy: 0.8712
Epoch 4/10
400/400 [==============================] - 453s 1s/step - loss: 0.4034 - accuracy: 0.8551 - val_loss: 0.3227 - val_accuracy: 0.8741
Epoch 5/10
400/400 [==============================] - 448s 1s/step - loss: 0.3302 - accuracy: 0.8771 - val_loss: 0.2587 - val_accuracy: 0.8994
Epoch 6/10
400/400 [==============================] - 441s 1s/step - loss: 0.2938 - accuracy: 0.8882 - val_loss: 0.2385 - val_accuracy: 0.9028
Epoch 7/10
400/400 [==============================] - 452s 1s/step - loss: 0.2552 - accuracy: 0.8976 - val_loss: 0.1991 - val_accuracy: 0.9125

In [31]:
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy * 100:.2f}%")

Test Loss: 0.1677391678094864, Test Accuracy: 92.60%


In [33]:
model.save('Emotion_Detection.h5')